# Hands On: Calculate Number of Tropical Nights
In this notebook you will:
- Read the saved selection and open the two files, which are needed for the task
- Calculate the number of tropical nights for both years
- Visualize the results on a map. You can use your prefered visualization package or stick to the example in the demo `use-case_frost_days_intake_xarray_cmip6.ipynb`

In [ ]:
import intake                         # to find data in a catalog, this notebook explains how it works
import xarray as xr
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point

## <font color='green'>Excercise: Read Your Collection</font> 
- use `pd.read_csv(<file location>/<file name>` to read your collection

In [ ]:
my_col = pd.read_csv('my_tasmin_collection.csv')

my_col

## <font color='green'>Exercise: Open Data File</font>
- open the data file, which contains the most recent year (2014)
- open a data file, which contains an arbitrary year in the past

You can open a data set with `xr.open_dataset(<path>)`

You can open a variable with `["<varname>"]`

In [ ]:
# Open data file
ds_recent = xr.open_dataset(my_col['path'][8])
ds_past   = xr.open_dataset(my_col['path'][3])

# Open variable "tasmin" over the whole time range
ds_recent_tasmin = ds_recent["tasmin"]
ds_past_tasmin = ds_past["tasmin"]

## <font color='green'>Look at both Data Files</font> 

In [ ]:
# Data file 1
ds_recent_tasmin

In [ ]:
# Data file 2
ds_past_tasmin

## <font color='green'>Excercise: Select Year from Data Files</font>
As you can see in the `time_range` column, each data file contains a range of 20 year years.
- Select in both data files the year which is needed.

You can select a year with `.sel(time='<year>')`

In [ ]:
ds_recent_tasmin_year = ds_recent_tasmin.sel(time='2014')
ds_past_tasmin_year   = ds_past_tasmin.sel(time='1915')

## <font color='green'>Excercise: Count Number of Tropical Nights</font>
- Count the number of tropical nights

The definition of a tropical night varies from region to region. According to the [German Weather Service](https://www.dwd.de/DE/service/lexikon/Functions/glossar.html?lv2=102672&lv3=102802), "a tropical night is a night in which the minimum air temperature is at least 20.0 °C".

In [ ]:
ds_recent_tasmin_year_count = ds_recent_tasmin_year.where(ds_recent_tasmin_year > 293.15).count(dim='time')
ds_past_tasmin_year_count = ds_past_tasmin_year.where(ds_past_tasmin_year > 293.15).count(dim='time')

Before plotting, a cyclic point has to be added, otherwise there will be a gap at the prime meridian

In [ ]:
lon_recent = ds_recent_tasmin_year_count.lon
lat_recent = ds_recent_tasmin_year_count.lat

lon_past = ds_past_tasmin_year_count.lon
lat_past = ds_past_tasmin_year_count.lat

ds_recent_tasmin_year_count, lon_recent = add_cyclic_point(ds_recent_tasmin_year_count, lon_recent)
ds_past_tasmin_year_count, lon_past = add_cyclic_point(ds_past_tasmin_year_count, lon_past)

## <font color='green'>Excercise: Visualization</font>
- Visualize the number of tropical nights for both years
- Create a third visualization, which is showing the difference between those two years
- Where do the numbers increase/decrease?

In [ ]:
# Plot 1

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())

plt.contourf(lon_recent, lat_recent, ds_recent_tasmin_year_count, 60,
            transform=ccrs.PlateCarree(),
            cmap='Reds')
ax.coastlines()
ax.set_global()

# Add a color bar
plt.colorbar(ax=ax)
plt.title('Number of Tropical Nights in Year 2014')
plt.show()

In [ ]:
# Plot 2

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())

plt.contourf(lon_past, lat_past, ds_past_tasmin_year_count, 60,
            transform=ccrs.PlateCarree(),
            cmap='Reds')
ax.coastlines()
ax.set_global()

# Add a color bar
plt.colorbar(ax=ax)
plt.title('Number of Tropical Nights in Year 1915')
plt.show()

In [ ]:
# Calculate the difference between both years

diff = ds_recent_tasmin_year_count -ds_past_tasmin_year_count

In [ ]:
# Difference plot

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mollweide())

levels = np.linspace(-60, 60, 61)

plt.contourf(lon_past, lat_past, diff, levels=levels,
            transform=ccrs.PlateCarree(),
            cmap='bwr')
ax.coastlines()
ax.set_global()

# Add a color bar
plt.colorbar(ax=ax)
plt.title('Difference in Number of Tropical Nights (Year 2014 - Year 1915)')
plt.show()